In [2]:
import numpy as np
import matplotlib.pyplot as plt
from trace_IO import *
from OptimumFilter import *

fs = 3906250
sigma = 0.7674
thresholds = [round(n * sigma, 4) for n in [3, 4, 5]]
energy_values = list(range(5, 101, 5))
pass_rates_by_threshold = {th: [] for th in thresholds}
n_sets = 100

# Load templates and noise PSD
qp_template = np.load("/home/dwong/DELight_mtr/trigger_study/wk15/templates/vac_ch_template.npy")
noise_psd = np.load("/home/dwong/DELight_mtr/templates/noise_psd_from_MMC.npy")

vac_of = OptimumFilter(qp_template, noise_psd, fs)


In [2]:
energy = 5
trace_path = f"/ceph/dwong/trigger_samples/lts/traces_energy_{energy}.zst"
loaded_traces = load_traces_from_zstd(trace_path, n_traces=n_sets)
example = loaded_traces[91][0][:32768]

In [3]:
type(example[0])

numpy.float32

In [4]:
import time

start_time = time.time()
ampl = vac_of.fit(example)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")


Time taken: 0.001729 seconds


In [2]:
energy = 5
trace_path = f"/ceph/dwong/trigger_samples/lts/traces_energy_{energy}.zst"
loaded_traces = load_traces_from_zstd(trace_path, n_traces=n_sets)

In [4]:
loaded_traces.shape

(100, 9, 250000)

In [6]:
example = loaded_traces[91][0]

window_size = 32768
num_windows = len(example) - window_size + 1

def fit_window(i):
    segment = example[i:i+window_size]
    return vac_of.fit(segment)


ampl_arr = []
chisq_arr = []
for i in range(num_windows):
    ampl = fit_window(i)
    ampl_arr.append(ampl)
ampl_arr = np.array(ampl_arr)


In [3]:
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
import multiprocessing

from trace_IO import *
from OptimumFilter import *
# ----------------------
# Constants & Parameters
# ----------------------
energy = 0
n_sets = 100
window_size = 32768
num_traces = 9
trace_length = 250_000
num_windows = trace_length - window_size + 1

trace_path = f"/ceph/dwong/trigger_samples/lts/traces_energy_{energy}.zst"
output_path = Path(f"/ceph/dwong/trigger_samples/lts/ampl_energy_{energy}.zst")

template_path = "/home/dwong/DELight_mtr/trigger_study/wk15/templates/vac_ch_template.npy"
noise_psd_path = "/home/dwong/DELight_mtr/templates/noise_psd_from_MMC.npy"
fs = 3906250

# ----------------------
# Load template and PSD once
# ----------------------
template = np.load(template_path)
noise_psd = np.load(noise_psd_path)

# ----------------------
# Process function (1 repeat = 9 traces)
# ----------------------
def process_one_repeat(traces, template, noise_psd, fs):
    vac = OptimumFilter(template, noise_psd, fs)
    result = np.empty((num_traces, num_windows), dtype=np.float32)
    window = np.empty(window_size, dtype=traces.dtype)

    for ch in range(num_traces):
        trace = traces[ch]
        ampl_arr = result[ch]
        for i in range(num_windows):
            window[:] = trace[i:i+window_size]
            ampl_arr[i] = vac.fit(window)

    return result

# ----------------------
# Main
# ----------------------
def main():
    # Load traces from file
    loaded_traces = load_traces_from_zstd(trace_path, n_traces=n_sets)  # shape: (100, 9, 250000)
    print(loaded_traces.shape)
    assert loaded_traces.shape == (n_sets, num_traces, trace_length), "Trace shape mismatch"

    # Use all available CPU cores
    n_jobs = multiprocessing.cpu_count()-2

    # Parallel amplitude extraction
    all_ampls = Parallel(n_jobs=n_jobs, backend="loky")(
        delayed(process_one_repeat)(loaded_traces[i], template, noise_psd, fs)
        for i in tqdm(range(n_sets), desc=f"Processing Energy {energy} ({n_sets} Traces)")
    )

    # Save results
    save_ampl_to_zstd(all_ampls, output_path)

if __name__ == "__main__":
    main()


(100, 9, 250000)


Processing Energy 0 (100 Traces): 100%|██████████| 100/100 [00:00<00:00, 3991.88it/s]
